Ritonavir(DB00503)：CN weights epoch:13

In [ ]:
# -*- coding: utf-8 -*-
import argparse
import numpy as np
import torch
import torch.nn as nn
import csv
from model_case import FusionLayer1, Model1, Model2, Model3, Model4

from collections import defaultdict
import os
import random
import warnings
import heapq
import math

# ===================== 0) Speed switches =====================
FAST_MODE = True
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
warnings.filterwarnings("ignore")

# ===================== 1) Args =====================
parser = argparse.ArgumentParser(description="GNN based on the whole datas")
parser.add_argument("--epoches", type=int, choices=[100, 500, 1000, 2000], default=30)
parser.add_argument("--batch_size", type=int, choices=[2048, 1024, 512, 256, 128], default=1024)
parser.add_argument("--weigh_decay", type=float, choices=[1e-1, 1e-2, 1e-3, 1e-4, 1e-8], default=1e-8)
parser.add_argument("--lr", type=float, choices=[1e-3, 1e-4, 1e-5, 4 * 1e-3], default=1 * 1e-3)
parser.add_argument("--neighbor_sample_size", choices=[4, 6, 10, 16], type=int, default=6)
parser.add_argument("--event_num", type=int, default=4)
parser.add_argument("--n_drug", type=int, default=1416)
parser.add_argument("--seed", type=int, default=1)
parser.add_argument("--dropout", type=float, default=0.5)
parser.add_argument("--embedding_num", type=int, choices=[128, 64, 256, 32], default=128)
parser.add_argument("--sample", type=int, choices=[1, 2, 3, 4, 5, 6], default=1)

# 原 score = dist / (n_masked ** alpha) 仍保留参数，但本版 Stage-1 默认用 z-score
parser.add_argument("--alpha", type=float, default=0.4)

# Stage-2
parser.add_argument("--top_k", type=int, default=5)
parser.add_argument("--max_tail_pairs", type=int, default=50000)  # <=0 means no limit (may be very slow)

# ===================== Z-score calibration controls (NEW) =====================
parser.add_argument("--use_zscore", action="store_true", default=True, help="Use z-score normalized score in Stage-1")
parser.add_argument("--bucket_width", type=int, default=10, help="n_masked bucket width for z-score stats")
parser.add_argument("--calib_min_count", type=int, default=20, help="min samples in a bucket; otherwise fallback to global stats")
parser.add_argument("--calib_passes", type=int, default=1, help="how many calibration passes (usually 1 is enough)")
parser.add_argument("--calib_max_pairs", type=int, default=-1, help="limit number of pairs for calibration; <=0 means all")

mask_id1 = -1.0

# ===================== Mask controls =====================
# Stage-1：支持“全局一个” + “按模态分别一个”
parser.add_argument("--mask_stage1", type=float, default=-1.0)  # 兜底：如果不单独设置 m1~m4，就用它
parser.add_argument("--mask_stage1_m1", type=float, default=-1.0)
parser.add_argument("--mask_stage1_m2", type=float, default=mask_id1)
parser.add_argument("--mask_stage1_m3", type=float, default=mask_id1)
parser.add_argument("--mask_stage1_m4", type=float, default=mask_id1)

# Stage-2：按模态分别一个（你原来的设定）
parser.add_argument("--mask_stage2_m1", type=float, default=50.0)
parser.add_argument("--mask_stage2_m2", type=float, default=-1.0)
parser.add_argument("--mask_stage2_m3", type=float, default=-1.0)
parser.add_argument("--mask_stage2_m4", type=float, default=1.8)

args, unknown = parser.parse_known_args()

# ===================== 2) Constants =====================
case_id = 1

path_prefix = f"../../datasets/1409/"
in_channels = 768

# 固定：药物A只用模态1；药物B从 1-4 中选
A_MODAL_FIXED = 1
B_MODAL_CANDIDATES = [1, 2, 3, 4]

# Stage-1 mask by modality（每个模态独立）
MASK_STAGE1_BY_MODAL = {
    1: float(args.mask_stage1_m1) if args.mask_stage1_m1 is not None else float(args.mask_stage1),
    2: float(args.mask_stage1_m2) if args.mask_stage1_m2 is not None else float(args.mask_stage1),
    3: float(args.mask_stage1_m3) if args.mask_stage1_m3 is not None else float(args.mask_stage1),
    4: float(args.mask_stage1_m4) if args.mask_stage1_m4 is not None else float(args.mask_stage1),
}

# Stage-2 mask by modality（原逻辑）
MASK_STAGE2_BY_MODAL = {
    1: float(args.mask_stage2_m1),
    2: float(args.mask_stage2_m2),
    3: float(args.mask_stage2_m3),
    4: float(args.mask_stage2_m4),
}

# ===================== 3) Random / determinism =====================
random.seed(args.seed)
os.environ["PYTHONHASHSEED"] = str(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(args.seed)
    torch.cuda.manual_seed_all(args.seed)

if FAST_MODE:
    torch.use_deterministic_algorithms(False)
    torch.backends.cudnn.enabled = True
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.deterministic = False
    try:
        torch.set_float32_matmul_precision("high")
    except Exception:
        pass
else:
    torch.use_deterministic_algorithms(True)
    os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.enabled = False

# ===================== 4) Data utils =====================
def read_dataset(drug_name_id, num):
    kg = defaultdict(list)
    tails = {}
    tail_id2name = {}
    relations = {}
    relation_id2name = {}

    filename = path_prefix + "dataset" + str(num) + ".txt"
    with open(filename, encoding="utf8") as reader:
        for line in reader:
            head, tail, relation = line.rstrip().split(",", 2)

            if tail not in tails:
                new_id = len(tails)
                tails[tail] = new_id
                tail_id2name[new_id] = tail

            if relation not in relations:
                new_id = len(relations)
                relations[relation] = new_id
                relation_id2name[new_id] = relation

            kg[drug_name_id[head]].append((tails[tail], relations[relation]))

    return kg, len(tails), len(relations), tail_id2name, relation_id2name

# ===================== 5) Read global DB_DDI + case DB map =====================
drugA, drugB, label = [], [], []
with open(path_prefix + "DB_DDI.csv", "r", encoding="utf-8", newline="") as f:
    reader = csv.DictReader(f)
    for row in reader:
        drugA.append(row["drug_A"].strip())
        drugB.append(row["drug_B"].strip())
        label.append(int(row["DDI"].strip()))
label = np.array(label)

with open(f"../../datasets/case_study/case{case_id}/DB_case.txt", "r", encoding="utf-8") as f:
    db_list = [line.strip() for line in f if line.strip()]

dict1 = {name: idx for idx, name in enumerate(db_list)}
id2drug = {drug_id: name for name, drug_id in dict1.items()}

# ===================== 6) Load 4 KGs =====================
dataset1_kg, dataset1_tail_len, dataset1_relation_len, tail_id2name1, relation_id2name1 = read_dataset(dict1, 1)
dataset2_kg, dataset2_tail_len, dataset2_relation_len, tail_id2name2, relation_id2name2 = read_dataset(dict1, 2)
dataset3_kg, dataset3_tail_len, dataset3_relation_len, tail_id2name3, relation_id2name3 = read_dataset(dict1, 3)
dataset4_kg, dataset4_tail_len, dataset4_relation_len, tail_id2name4, relation_id2name4 = read_dataset(dict1, 4)

dataset = {"dataset1": dataset1_kg, "dataset2": dataset2_kg, "dataset3": dataset3_kg, "dataset4": dataset4_kg}

# ===================== 7) Build feats + edge_index =====================
drug_n = args.n_drug
feats_1 = np.full((drug_n, dataset1_tail_len), -1.0, dtype=np.float32)
feats_2 = np.full((drug_n, dataset2_tail_len), -1.0, dtype=np.float32)
feats_3 = np.full((drug_n, dataset3_tail_len), -1.0, dtype=np.float32)
feats_4 = np.full((drug_n, dataset4_tail_len), -1.0, dtype=np.float32)

edge_index_1, edge_index_2, edge_index_3, edge_index_4 = ([[], []], [[], []], [[], []], [[], []])
real_edge_1, real_edge_2, real_edge_3, real_edge_4 = [], [], [], []
num_classes_1 = []

for p, kg_name in enumerate(dataset):
    kg = dataset[kg_name]
    for i in kg.keys():
        for t_id, rel_id in kg[i]:
            if p == 0:
                feats_1[i][t_id] = float(rel_id)
                edge_index_1[0].append(i)
                edge_index_1[1].append(drug_n + t_id)
                real_edge_1.append(rel_id)
                num_classes_1.append(rel_id)
            elif p == 1:
                feats_2[i][t_id] = 1.0
                edge_index_2[0].append(i)
                edge_index_2[1].append(drug_n + t_id)
                real_edge_2.append(1)
            elif p == 2:
                feats_3[i][t_id] = 1.0
                edge_index_3[0].append(i)
                edge_index_3[1].append(drug_n + t_id)
                real_edge_3.append(1)
            elif p == 3:
                feats_4[i][t_id] = 1.0
                edge_index_4[0].append(i)
                edge_index_4[1].append(drug_n + t_id)
                real_edge_4.append(1)

edge_index_1 = torch.tensor(edge_index_1, dtype=torch.long, device=device)
edge_index_2 = torch.tensor(edge_index_2, dtype=torch.long, device=device)
edge_index_3 = torch.tensor(edge_index_3, dtype=torch.long, device=device)
edge_index_4 = torch.tensor(edge_index_4, dtype=torch.long, device=device)

real_edge_1 = torch.tensor(real_edge_1, dtype=torch.long, device=device)
real_edge_2 = torch.tensor(real_edge_2, dtype=torch.long, device=device)
real_edge_3 = torch.tensor(real_edge_3, dtype=torch.long, device=device)
real_edge_4 = torch.tensor(real_edge_4, dtype=torch.long, device=device)

num_classes_1 = set(num_classes_1)

# ===================== 8) Modal config =====================
feats_t1 = torch.from_numpy(feats_1).to(device)
feats_t2 = torch.from_numpy(feats_2).to(device)
feats_t3 = torch.from_numpy(feats_3).to(device)
feats_t4 = torch.from_numpy(feats_4).to(device)

modal_cfg = {
    1: dict(
        kg=dataset1_kg,
        tail_id2name=tail_id2name1,
        rel_id2name=relation_id2name1,
        feats=feats_t1,
        tail_len=dataset1_tail_len,
        edge_index=edge_index_1,
        real_edge=real_edge_1,
        model_cls=Model1,
        hidden_dim=256,
        out_dim=len(num_classes_1),
    ),
    2: dict(
        kg=dataset2_kg,
        tail_id2name=tail_id2name2,
        rel_id2name=relation_id2name2,
        feats=feats_t2,
        tail_len=dataset2_tail_len,
        edge_index=edge_index_2,
        real_edge=real_edge_2,
        model_cls=Model2,
        hidden_dim=args.embedding_num,
        out_dim=2,
    ),
    3: dict(
        kg=dataset3_kg,
        tail_id2name=tail_id2name3,
        rel_id2name=relation_id2name3,
        feats=feats_t3,
        tail_len=dataset3_tail_len,
        edge_index=edge_index_3,
        real_edge=real_edge_3,
        model_cls=Model3,
        hidden_dim=256,
        out_dim=2,
    ),
    4: dict(
        kg=dataset4_kg,
        tail_id2name=tail_id2name4,
        rel_id2name=relation_id2name4,
        feats=feats_t4,
        tail_len=dataset4_tail_len,
        edge_index=edge_index_4,
        real_edge=real_edge_4,
        model_cls=Model4,
        hidden_dim=args.embedding_num,
        out_dim=2,
    ),
}

active_modalities = [1, 2, 3, 4]

# ===================== 9) Faster caches =====================
_tail_list_cache = {m: {} for m in active_modalities}
_rel_map_cache = {m: {} for m in active_modalities}

def get_tail_list(drug_id, modal_id):
    cache = _tail_list_cache[modal_id]
    if drug_id in cache:
        return cache[drug_id]
    kg = modal_cfg[modal_id]["kg"]
    lst = [t_id for (t_id, _rel) in kg.get(drug_id, [])]
    cache[drug_id] = lst
    return lst

def get_rel_map(drug_id, modal_id):
    cache = _rel_map_cache[modal_id]
    if drug_id in cache:
        return cache[drug_id]
    kg = modal_cfg[modal_id]["kg"]
    rel_id2name = modal_cfg[modal_id]["rel_id2name"]
    d = defaultdict(set)
    for t_id, rel_id in kg.get(drug_id, []):
        d[t_id].add(rel_id2name[rel_id].lower())
    cache[drug_id] = d
    return d

# ===================== 10) Build net =====================
def build_net(feats_dict, active_modalities):
    modules = []
    for m_id in active_modalities:
        cfg = modal_cfg[m_id]
        modules.append(
            cfg["model_cls"](
                args,
                cfg["tail_len"],
                feats_dict[m_id],
                cfg["edge_index"],
                in_channels,
                args.embedding_num,
                args.embedding_num,
                cfg["hidden_dim"],
                cfg["out_dim"],
                cfg["real_edge"],
            )
        )
    modules.append(FusionLayer1(args))
    return nn.Sequential(*modules)

# ===================== 11) Find feats tensor =====================
def _iter_all_modules(root: nn.Module):
    yield root
    for m in root.modules():
        yield m

def find_feat_tensor(mod: nn.Module, expected_shape):
    common_names = ["feats", "feat", "features", "x", "X", "input_feat", "input_feats"]
    for name in common_names:
        if hasattr(mod, name):
            v = getattr(mod, name)
            if torch.is_tensor(v) and tuple(v.shape) == tuple(expected_shape):
                return v, f"{mod.__class__.__name__}.{name}"

    for n, b in mod.named_buffers(recurse=True):
        if torch.is_tensor(b) and tuple(b.shape) == tuple(expected_shape):
            return b, f"buffer:{n}"
    for n, p in mod.named_parameters(recurse=True):
        if torch.is_tensor(p) and tuple(p.shape) == tuple(expected_shape):
            return p, f"param:{n}"

    for m in _iter_all_modules(mod):
        for k, v in vars(m).items():
            if torch.is_tensor(v) and tuple(v.shape) == tuple(expected_shape):
                return v, f"attr:{m.__class__.__name__}.{k}"

    raise RuntimeError(f"[ERROR] Cannot find feats tensor in {mod.__class__.__name__} with shape={expected_shape}.")

# ===================== 12) Prepare two nets: full + mask =====================
checkpoint_path = f"../../datasets/case_study/case{case_id}/case{case_id}_weight"
checkpoint = torch.load(checkpoint_path, map_location=device)

base_feats_dict_full = {m_id: modal_cfg[m_id]["feats"] for m_id in active_modalities}
base_feats_dict_mask = {m_id: modal_cfg[m_id]["feats"].clone() for m_id in active_modalities}

net_full = build_net(base_feats_dict_full, active_modalities).to(device)
net_mask = build_net(base_feats_dict_mask, active_modalities).to(device)

net_full.load_state_dict(checkpoint["model_state_dict"])
net_mask.load_state_dict(checkpoint["model_state_dict"])
net_full.eval()
net_mask.eval()

mod_idx = {m_id: i for i, m_id in enumerate(active_modalities)}

mask_feat_ref = {}
mask_feat_ref_name = {}
for m_id in active_modalities:
    m = net_mask[mod_idx[m_id]]
    exp_shape = base_feats_dict_mask[m_id].shape
    t, where = find_feat_tensor(m, exp_shape)
    mask_feat_ref[m_id] = t
    mask_feat_ref_name[m_id] = where

print("[INFO] Located feats in net_mask:")
for m_id in active_modalities:
    print(f"  modality {m_id}: {mask_feat_ref_name[m_id]}  shape={tuple(mask_feat_ref[m_id].shape)}")

# ===================== 13) Read case-study pairs CSV =====================
pairs = []
total_cases = 0
with open(f"../../datasets/case_study/case{case_id}/case{case_id}_DB_DDI_Test.csv", "r", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    for row in reader:
        drugA_db = row["drug_A"].strip()
        drugB_db = row["drug_B"].strip()
        ddi_label = row.get("DDI", "").strip()
        pairs.append((drugA_db, drugB_db, ddi_label))
        total_cases += 1
print("Number of samples read:", len(pairs))

# ===================== 14) In-place mask helpers =====================
def mask_row_inplace(feat_mat: torch.Tensor, row_id: int, mask_value: float):
    backup = feat_mat[row_id].clone()
    feat_mat[row_id].fill_(float(mask_value))
    return backup

def restore_row_inplace(feat_mat: torch.Tensor, row_id: int, backup: torch.Tensor):
    feat_mat[row_id].copy_(backup)

def mask_one_inplace(feat_mat: torch.Tensor, row_id: int, col_id: int, mask_value: float):
    backup = feat_mat[row_id, col_id].clone()
    feat_mat[row_id, col_id] = float(mask_value)
    return backup

def restore_one_inplace(feat_mat: torch.Tensor, row_id: int, col_id: int, backup: torch.Tensor):
    feat_mat[row_id, col_id] = backup

# ===================== 15) Forward (only embedding) =====================
@torch.inference_mode()
def forward_full_once(test_x_gpu: torch.Tensor):
    out = net_full(test_x_gpu)
    return out[-1]

@torch.inference_mode()
def forward_mask_once(test_x_gpu: torch.Tensor):
    out = net_mask(test_x_gpu)
    return out[-1]

# ===================== 15.5) Z-score stats (NEW) =====================
EPS = 1e-12
BUCKET_WIDTH = int(args.bucket_width)
CALIB_MIN_COUNT = int(args.calib_min_count)

class RunningStats:
    def __init__(self):
        self.n = 0
        self.mean = 0.0
        self.M2 = 0.0

    def update(self, x: float):
        self.n += 1
        delta = x - self.mean
        self.mean += delta / self.n
        delta2 = x - self.mean
        self.M2 += delta * delta2

    def std(self) -> float:
        if self.n < 2:
            return 0.0
        var = self.M2 / (self.n - 1)
        return math.sqrt(max(var, 0.0))

def bucket_n(n_masked: int) -> int:
    return int(round(n_masked / BUCKET_WIDTH) * BUCKET_WIDTH)

dist_stats_by_bucket = defaultdict(RunningStats)
global_dist_stats = RunningStats()

def zscore(dist: float, n_masked: int) -> float:
    b = bucket_n(n_masked)
    st = dist_stats_by_bucket.get(b, None)
    if st is None or st.n < CALIB_MIN_COUNT:
        mu = global_dist_stats.mean
        sd = global_dist_stats.std()
    else:
        mu = st.mean
        sd = st.std()
    return (dist - mu) / (sd + EPS)

# ===================== 16) Stats =====================
best_B_count = defaultdict(int)
score_sum_by_B = defaultdict(float)
score_cnt_by_B = defaultdict(int)

# ===================== 16.5) Stage-2 Top1 Stats =====================
stage2_total_effective = 0  # 有跑出 Stage-2 且产生 top1 的样本数

# key = (best_mB, featureA_name, relA_tuple, featureB_name, relB_tuple)
stage2_top1_pair_count = defaultdict(int)
stage2_top1_pair_dist_sum = defaultdict(float)

# 可选：按 B 模态分开统计
stage2_top1_pair_count_by_B = {m: defaultdict(int) for m in [1, 2, 3, 4]}
stage2_top1_pair_dist_sum_by_B = {m: defaultdict(float) for m in [1, 2, 3, 4]}
stage2_total_by_B = defaultdict(int)

# ===================== 17) Stage-2 cap helper =====================
def maybe_subsample_tail_lists(tailsA, tailsB, max_pairs: int):
    if max_pairs is None or max_pairs <= 0:
        return tailsA, tailsB
    nA, nB = len(tailsA), len(tailsB)
    if nA == 0 or nB == 0:
        return tailsA, tailsB
    if nA * nB <= max_pairs:
        return tailsA, tailsB

    sizeA = min(nA, max(1, int(math.sqrt(max_pairs))))
    sizeB = min(nB, max(1, max_pairs // sizeA))
    if sizeA * sizeB > max_pairs:
        sizeB = max(1, max_pairs // sizeA)

    subA = random.sample(tailsA, sizeA) if sizeA < nA else tailsA
    subB = random.sample(tailsB, sizeB) if sizeB < nB else tailsB
    return subA, subB

# ===================== 18) Stage-1 dist evaluator (NEW helper) =====================
def compute_stage1_dists(drugA_id: int, drugB_id: int, test_x_gpu: torch.Tensor, Emb_full: torch.Tensor):
    # A 固定模态1
    A_cnt = len(get_tail_list(drugA_id, A_MODAL_FIXED))
    if A_cnt == 0:
        return None, None, None

    # B 可选四个模态（仅保留有节点的）
    B_candidates = []
    B_tail_counts = {}
    for mB in B_MODAL_CANDIDATES:
        c = len(get_tail_list(drugB_id, mB))
        B_tail_counts[mB] = c
        if c > 0:
            B_candidates.append(mB)
    if not B_candidates:
        return A_cnt, B_tail_counts, []

    featA = mask_feat_ref[A_MODAL_FIXED]
    maskA1 = MASK_STAGE1_BY_MODAL[A_MODAL_FIXED]

    rows = []  # list of (mB, dist, n_masked)
    for mB in B_candidates:
        n_masked = A_cnt + B_tail_counts[mB]
        if n_masked <= 0:
            continue

        featB = mask_feat_ref[mB]
        maskB1 = MASK_STAGE1_BY_MODAL[mB]

        backupA = mask_row_inplace(featA, drugA_id, maskA1)
        backupB = mask_row_inplace(featB, drugB_id, maskB1)

        Emb_mask = forward_mask_once(test_x_gpu)

        restore_row_inplace(featA, drugA_id, backupA)
        restore_row_inplace(featB, drugB_id, backupB)

        dist = torch.norm(Emb_full - Emb_mask, p=2, dim=1).item()
        rows.append((mB, dist, n_masked))
    return A_cnt, B_tail_counts, rows

# ===================== 18.1) Calibration pass (NEW) =====================
def run_calibration(pairs):
    # 可选：限制 calibration 用多少对
    pairs_use = pairs
    if args.calib_max_pairs is not None and int(args.calib_max_pairs) > 0:
        pairs_use = pairs[: int(args.calib_max_pairs)]

    total_rows = 0
    skipped = 0
    for _pass in range(int(args.calib_passes)):
        for (drugA_db, drugB_db, _ddi_label) in pairs_use:
            if drugA_db not in dict1 or drugB_db not in dict1:
                skipped += 1
                continue

            drugA_id = dict1[drugA_db]
            drugB_id = dict1[drugB_db]
            test_x_gpu = torch.tensor([[drugA_id, drugB_id]], dtype=torch.long, device=device)

            Emb_full = forward_full_once(test_x_gpu)
            A_cnt, B_tail_counts, rows = compute_stage1_dists(drugA_id, drugB_id, test_x_gpu, Emb_full)
            if rows is None or len(rows) == 0:
                skipped += 1
                continue

            for (mB, dist, n_masked) in rows:
                b = bucket_n(n_masked)
                dist_stats_by_bucket[b].update(dist)
                global_dist_stats.update(dist)
                total_rows += 1

    print("\n[CALIB] Done.")
    print(f"[CALIB] total dist rows collected: {total_rows}, skipped pairs: {skipped}")
    print(f"[CALIB] global mean={global_dist_stats.mean:.6f}, global std={global_dist_stats.std():.6f}")
    # 打印几个桶（可选）
    top_buckets = sorted(dist_stats_by_bucket.items(), key=lambda x: x[1].n, reverse=True)[:10]
    if top_buckets:
        print("[CALIB] top buckets by count:")
        for b, st in top_buckets:
            print(f"  bucket n={b}: count={st.n}, mean={st.mean:.6f}, std={st.std():.6f}")

# ===================== 18.2) Core analysis =====================
def analyze_drug_pair(drugA_db, drugB_db):
    global stage2_total_effective

    if drugA_db not in dict1 or drugB_db not in dict1:
        print(f"  [Skip] {drugA_db}, {drugB_db} not found in dict1.")
        return

    drugA_id = dict1[drugA_db]
    drugB_id = dict1[drugB_db]
    test_x_gpu = torch.tensor([[drugA_id, drugB_id]], dtype=torch.long, device=device)

    Emb_full = forward_full_once(test_x_gpu)

    A_cnt, B_tail_counts, rows = compute_stage1_dists(drugA_id, drugB_id, test_x_gpu, Emb_full)
    if rows is None:
        print("  [Skip] DrugA has no tails in modality 1.")
        return
    if rows == []:
        print("  [Skip] DrugB has no tails in any modality (1-4) OR no valid rows.")
        return

    # ---------- Stage 1: pick best mB ----------
    alpha = float(args.alpha)
    best = None
    best_score = -1e18

    for (mB, dist, n_masked) in rows:
        if args.use_zscore:
            score = zscore(dist, n_masked)
        else:
            score = dist / ((float(n_masked) ** alpha) + 1e-12)

        if score > best_score:
            best_score = score
            best = (score, dist, mB, n_masked)

    if best is None:
        print("  [Warning] No valid B modality found.")
        return

    best_score, best_dist, best_mB, best_n_masked = best
    best_B_count[best_mB] += 1
    score_sum_by_B[best_mB] += best_score
    score_cnt_by_B[best_mB] += 1

    print(
        f"  Best modality pair (A fixed): A->{A_MODAL_FIXED}, B->{best_mB}, "
        f"score={best_score:.6f} ({'zscore' if args.use_zscore else f'alpha={alpha}'}), "
        f"dist={best_dist:.6f}, n_masked={best_n_masked}, "
        f"maskA1={MASK_STAGE1_BY_MODAL[A_MODAL_FIXED]}, maskB1={MASK_STAGE1_BY_MODAL[best_mB]}"
    )

    # ---------- Stage 2: tail-level top-k ----------
    tailsA = get_tail_list(drugA_id, A_MODAL_FIXED)
    tailsB = get_tail_list(drugB_id, best_mB)
    if not tailsA or not tailsB:
        print("  [Info] Empty tail list in Stage-2.")
        return

    tailsA_use, tailsB_use = maybe_subsample_tail_lists(tailsA, tailsB, args.max_tail_pairs)
    if len(tailsA_use) * len(tailsB_use) != len(tailsA) * len(tailsB):
        print(
            f"  [Info] Stage-2 subsampled: |A| {len(tailsA)}->{len(tailsA_use)}, "
            f"|B| {len(tailsB)}->{len(tailsB_use)}, pairs={len(tailsA_use)*len(tailsB_use)}"
        )

    relA_map = get_rel_map(drugA_id, A_MODAL_FIXED)
    relB_map = get_rel_map(drugB_id, best_mB)

    featA2 = mask_feat_ref[A_MODAL_FIXED]
    featB2 = mask_feat_ref[best_mB]

    maskA2 = MASK_STAGE2_BY_MODAL[A_MODAL_FIXED]
    maskB2 = MASK_STAGE2_BY_MODAL[best_mB]

    heap = []  # min-heap: (dist, nameA, relA_set, nameB, relB_set)
    top_k = int(args.top_k)

    for tA in tailsA_use:
        for tB in tailsB_use:
            bA = mask_one_inplace(featA2, drugA_id, tA, maskA2)
            bB = mask_one_inplace(featB2, drugB_id, tB, maskB2)

            Emb_mask = forward_mask_once(test_x_gpu)

            restore_one_inplace(featA2, drugA_id, tA, bA)
            restore_one_inplace(featB2, drugB_id, tB, bB)

            dist2 = torch.norm(Emb_full - Emb_mask, p=2, dim=1).item()

            nameA = modal_cfg[A_MODAL_FIXED]["tail_id2name"][tA]
            nameB = modal_cfg[best_mB]["tail_id2name"][tB]
            relA_names = relA_map.get(tA, set())
            relB_names = relB_map.get(tB, set())

            item = (dist2, nameA, relA_names, nameB, relB_names)
            if len(heap) < top_k:
                heapq.heappush(heap, item)
            else:
                if dist2 > heap[0][0]:
                    heapq.heapreplace(heap, item)

    if not heap:
        print("  [Info] No tail combinations produced in Stage-2.")
        return

    top_list = sorted(heap, key=lambda x: x[0], reverse=True)

    # ---- Stage-2 Top1 stats (只统计第一名) ----
    d1, nameA1, relA1, nameB1, relB1 = top_list[0]
    relA_key = tuple(sorted(relA1))
    relB_key = tuple(sorted(relB1))
    key = (best_mB, nameA1, relA_key, nameB1, relB_key)

    stage2_total_effective += 1
    stage2_top1_pair_count[key] += 1
    stage2_top1_pair_dist_sum[key] += float(d1)

    stage2_total_by_B[best_mB] += 1
    stage2_top1_pair_count_by_B[best_mB][key] += 1
    stage2_top1_pair_dist_sum_by_B[best_mB][key] += float(d1)

    print(
        f"  Stage-2 Top-{len(top_list)} feature combinations (A->1, B->{best_mB}) "
        f"[maskA2={maskA2}, maskB2={maskB2}]:"
    )
    for i, (d, nameA, relA, nameB, relB) in enumerate(top_list, 1):
        print(
            f"    [{i}] featureA={nameA} (relations={','.join(sorted(relA))}) | "
            f"featureB={nameB} (relations={','.join(sorted(relB))}) | "
            f"distance={d:.6f}"
        )

# ===================== 19) Run calibration + all pairs =====================
print(
    f"\n[INFO] A fixed modality = 1; B in {{1,2,3,4}}\n"
    f"[INFO] Stage-1 mask by modality = {MASK_STAGE1_BY_MODAL}\n"
    f"[INFO] Stage-2 mask by modality = {MASK_STAGE2_BY_MODAL}\n"
    f"[INFO] Stage-1 scoring = {'z-score by n_masked buckets' if args.use_zscore else 'dist / (n_masked ** alpha)'}\n"
    f"[INFO] zscore bucket_width={BUCKET_WIDTH}, calib_min_count={CALIB_MIN_COUNT}, calib_max_pairs={args.calib_max_pairs}\n"
    f"[INFO] Stage-2 top_k={args.top_k}, max_tail_pairs={args.max_tail_pairs}\n"
)

if args.use_zscore:
    run_calibration(pairs)

for idx, (drugA_db, drugB_db, ddi_label) in enumerate(pairs, 1):
    print(f"\n===== Case {idx}: {drugA_db} - {drugB_db} (DDI={ddi_label}) =====")
    analyze_drug_pair(drugA_db, drugB_db)

# ===================== 20) Report =====================
print("\n===== Summary Report =====")
print("Total number of samples:", total_cases)

print("\n--- Best B modality frequency (A fixed = 1) ---")
total_best = sum(best_B_count.values())
for mB in [1, 2, 3, 4]:
    c = best_B_count.get(mB, 0)
    rate = (c / total_best) if total_best > 0 else 0.0
    avg_score = (score_sum_by_B[mB] / score_cnt_by_B[mB]) if score_cnt_by_B.get(mB, 0) > 0 else 0.0
    print(f"Best B modality = {mB}: count={c} ({rate:.4f}), avg_score={avg_score:.6f}")

# ===================== 20.5) Stage-2 Top1 Report =====================
print("\n--- Stage-2 Top1 feature-pair hit report ---")
print(f"Stage-2 effective samples (have top1): {stage2_total_effective}")

if stage2_total_effective == 0:
    print("No Stage-2 top1 stats collected (all skipped or empty).")
else:
    TOPN = 20
    items = sorted(stage2_top1_pair_count.items(), key=lambda x: x[1], reverse=True)[:TOPN]

    print(f"\nTop-{len(items)} most frequent Stage-2 top1 feature pairs (global):")
    for rank, (k, cnt) in enumerate(items, 1):
        best_mB, nameA, relA_key, nameB, relB_key = k
        rate = cnt / stage2_total_effective
        avg_dist = stage2_top1_pair_dist_sum[k] / cnt
        print(
            f"  [{rank}] B_modal={best_mB} | "
            f"A={nameA} (rels={','.join(relA_key)}) | "
            f"B={nameB} (rels={','.join(relB_key)}) | "
            f"count={cnt} ({rate:.4f}) | avg_dist={avg_dist:.6f}"
        )

    print("\nTop1 feature pairs by best B modality:")
    for mB in [1, 2, 3, 4]:
        total_m = stage2_total_by_B.get(mB, 0)
        print(f"\n  [B modality = {mB}] total_top1_samples={total_m}")
        if total_m == 0:
            continue
        items_m = sorted(stage2_top1_pair_count_by_B[mB].items(), key=lambda x: x[1], reverse=True)[:TOPN]
        for rank, (k, cnt) in enumerate(items_m, 1):
            best_mB, nameA, relA_key, nameB, relB_key = k
            rate = cnt / total_m
            avg_dist = stage2_top1_pair_dist_sum_by_B[mB][k] / cnt
            print(
                f"    [{rank}] A={nameA} (rels={','.join(relA_key)}) | "
                f"B={nameB} (rels={','.join(relB_key)}) | "
                f"count={cnt} ({rate:.4f}) | avg_dist={avg_dist:.6f}"
            )


[INFO] Located feats in net_mask:
  modality 1: attr:GCNEncoder.drug_features  shape=(1416, 1333)
  modality 2: attr:GCNEncoder.drug_features  shape=(1416, 1024)
  modality 3: attr:GCNEncoder.drug_features  shape=(1416, 158)
  modality 4: attr:GCNEncoder.drug_features  shape=(1416, 397)
Number of samples read: 429

[INFO] A fixed modality = 1; B in {1,2,3,4}
[INFO] Stage-1 mask by modality = {1: -1.0, 2: -1.0, 3: -1.0, 4: -1.0}
[INFO] Stage-2 mask by modality = {1: 50.0, 2: -1.0, 3: -1.0, 4: 1.8}
[INFO] Stage-1 scoring = z-score by n_masked buckets
[INFO] zscore bucket_width=10, calib_min_count=20, calib_max_pairs=-1
[INFO] Stage-2 top_k=5, max_tail_pairs=50000


[CALIB] Done.
[CALIB] total dist rows collected: 1716, skipped pairs: 0
[CALIB] global mean=83.193230, global std=19.672382
[CALIB] top buckets by count:
  bucket n=20: count=449, mean=77.555272, std=14.533488
  bucket n=80: count=426, mean=94.005866, std=11.505160
  bucket n=70: count=302, mean=78.735857, std=11.602106
  buck